In [2]:
# Downloading packages
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,classification_report

In [4]:
# Read trainig and validation dataset
header_list = ["text","intent","buffer"]
train = pd.read_csv("https://raw.githubusercontent.com/T-I-P/Hope-Speech-Detection/master/English/english_hope_train.csv", '\t',header=None,names=header_list)
valid = pd.read_csv("https://raw.githubusercontent.com/T-I-P/Hope-Speech-Detection/master/English/english_hope_dev.csv", '\t',header=None,names=header_list)
train = train.drop(labels='buffer',axis=1)
valid = valid.drop(labels='buffer',axis=1)

In [5]:
train.head()

,text,intent
0,these tiktoks radiate gay chaotic energy and i...,Non_hope_speech
1,@Champions Again He got killed for using false...,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Bla...,Non_hope_speech
4,Whenever we say black isn't that racists? Why...,Non_hope_speech


In [6]:
train = train.loc[train['intent']!='not-English']
valid = valid.loc[valid['intent']!='not-English']

In [7]:
print(train.shape, valid.shape)
print(np.unique(train['intent']))

(22740, 2) (2841, 2)
['Hope_speech' 'Non_hope_speech']


### Data Cleaning

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have", "let 's": "let us", "'em": "them"}

In [10]:
pip install Unidecode

     |████████████████████████████████| 245kB 17.8MB/s 


In [14]:
# Code block for data cleaning
import codecs
import unidecode
import re
import spacy
nlp = spacy.load('en')

def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [12]:
train['text'] = [spacy_cleaner(t) for t in train.text]

In [13]:
valid['text'] = [spacy_cleaner(t) for t in valid.text]

In [17]:
# Change all the text to lower case. This is required as python interprets 'python', 'Python" and 'PYTHON' differently
train['text'] = [entry.lower() for entry in train['text']]
valid['text'] = [entry.lower() for entry in valid['text']]

In [21]:
train.head()

,text,intent
0,these tiktok radiate gay chaotic energy and lo...,Non_hope_speech
1,again he get kill for use false money,Non_hope_speech
2,it be not that all life do not matter,Non_hope_speech
3,be it really that difficult to understand blac...,Non_hope_speech
4,whenever we say black be not that racist why d...,Non_hope_speech


In [28]:
valid.isnull().sum()

text      0
intent    0
dtype: int64

In [29]:
train.to_csv('train_cleaned.csv')
valid.to_csv('valid_cleaned.csv')

## Tokenization / Lemmatization

In [31]:
# Tokenization : In this each entry in the corpus will be broken into set of words
train['text']= [word_tokenize(entry) for entry in train['text']]

In [34]:
# Tokenization : In this each entry in the corpus will be broken into set of words
valid['text']= [word_tokenize(entry) for entry in valid['text']]

In [32]:
# Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting on training dataset
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(train['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    train.loc[index,'text_final'] = str(Final_words)

In [35]:
# Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting on validation dataset
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(valid['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    valid.loc[index,'text_final'] = str(Final_words)

In [39]:
valid.isnull().sum()

text          2
intent        2
text_final    2
dtype: int64

In [43]:
train.dropna(subset=['intent'], inplace=True)
train.dropna(subset=['text_final'], inplace=True)
valid.dropna(subset=['intent'], inplace=True)
valid.dropna(subset=['text_final'], inplace=True)

In [45]:
train.isnull().sum()

text          0
intent        0
text_final    0
dtype: int64

In [46]:
train.head()

,text,intent,text_final
0,"[these, tiktok, radiate, gay, chaotic, energy,...",Non_hope_speech,"['tiktok', 'radiate', 'gay', 'chaotic', 'energ..."
1,"[again, he, get, kill, for, use, false, money]",Non_hope_speech,"['get', 'kill', 'use', 'false', 'money']"
2,"[it, be, not, that, all, life, do, not, matter]",Non_hope_speech,"['life', 'matter']"
3,"[be, it, really, that, difficult, to, understa...",Non_hope_speech,"['really', 'difficult', 'understand', 'black',..."
4,"[whenever, we, say, black, be, not, that, raci...",Non_hope_speech,"['whenever', 'say', 'black', 'racist', 'say', ..."


In [47]:
train.to_csv('train_tokenized.csv')
valid.to_csv('valid_tokenized.csv')